<a href="https://colab.research.google.com/github/adenurchalisa/Capstone-Model/blob/main/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import joblib
import os
from pathlib import Path

In [5]:
# Fungsi untuk mencari file model di beberapa lokasi potensial
def find_model_file(filename):
    # Daftar kemungkinan lokasi file model
    possible_locations = [
        # 1. Di folder yang sama dengan script ini
        os.path.join(os.path.dirname(__file__), filename),

        # 2. Di subfolder 'models'
        os.path.join(os.path.dirname(__file__), 'models', filename),

        # 3. Di folder parent/root project
        os.path.join(os.path.dirname(os.path.dirname(__file__)), 'models', filename),

        # 4. Path absolut untuk deployment
        os.path.join('/app/models', filename),
    ]

    # Cek setiap lokasi
    for location in possible_locations:
        if os.path.exists(location):
            return location

    # Jika file tidak ditemukan di lokasi manapun
    raise FileNotFoundError(f"Model file {filename} tidak ditemukan di lokasi manapun")

# Inisialisasi variabel global
model = None
low_error_users = []
high_error_users = []

# Load model dan data segmentasi user
try:
    model_path = find_model_file('xgb_spending_predictor.pkl')
    segments_path = find_model_file('user_segments.pkl')

    model = joblib.load(model_path)
    user_segments = joblib.load(segments_path)

    low_error_users = user_segments['low_error_users']
    high_error_users = user_segments['high_error_users']

    print(f"Model berhasil dimuat dari {model_path}")
    print(f"Data segmentasi user berhasil dimuat dari {segments_path}")

except Exception as e:
    print(f"Error saat memuat model: {e}")
    # Jika dalam development, biarkan error terus muncul
    # Jika dalam production, bisa tambahkan fallback

def predict_spending(user_id, latest_spending, rolling3_spending, target_month,
                   is_ramadan=0, is_lebaran=0, is_akhir_tahun=0):
    """
    Memprediksi pengeluaran user untuk bulan yang ditargetkan

    Parameters:
    -----------
    user_id : str
        ID unik pengguna
    latest_spending : float
        Total pengeluaran bulan terakhir
    rolling3_spending : float
        Rata-rata pengeluaran 3 bulan terakhir
    target_month : int
        Bulan yang ingin diprediksi (1-12)
    is_ramadan : int, optional
        Flag apakah bulan target adalah bulan Ramadan (0 atau 1)
    is_lebaran : int, optional
        Flag apakah bulan target adalah bulan Lebaran (0 atau 1)
    is_akhir_tahun : int, optional
        Flag untuk bulan Desember (default 1 jika target_month=12)

    Returns:
    --------
    dict
        Hasil prediksi dengan rentang dan tingkat keyakinan
    """
    # Verifikasi model sudah dimuat
    if model is None:
        return {
            'error': 'Model belum dimuat dengan benar',
            'user_id': user_id,
            'prediction': 0,
            'min_prediction': 0,
            'max_prediction': 0,
            'confidence_level': "none"
        }

    # Set is_akhir_tahun otomatis jika bulan = 12
    if target_month == 12 and is_akhir_tahun == 0:
        is_akhir_tahun = 1

    # Buat data input
    input_data = pd.DataFrame({
        'pengeluaran_bulan_lalu': [latest_spending],
        'rolling3_pengeluaran': [rolling3_spending],
        'bulan': [target_month],
        'is_ramadan': [is_ramadan],
        'is_lebaran': [is_lebaran],
        'is_akhir_tahun': [is_akhir_tahun]
    })

    try:
        # Prediksi dengan model
        pred_log = model.predict(input_data)[0]
        prediction = np.expm1(pred_log)

        # Buat rentang prediksi berdasarkan segmen user
        if user_id in high_error_users:
            confidence = "low"
            lower = prediction * 0.4  # -60%
            upper = prediction * 2.0  # +100%
        elif user_id in low_error_users:
            confidence = "high"
            lower = prediction * 0.8  # -20%
            upper = prediction * 1.2  # +20%
        else:
            confidence = "medium"
            lower = prediction * 0.6  # -40%
            upper = prediction * 1.4  # +40%

        # Format hasil untuk web
        return {
            'user_id': user_id,
            'prediction': int(prediction),
            'min_prediction': int(lower),
            'max_prediction': int(upper),
            'confidence_level': confidence
        }

    except Exception as e:
        print(f"Error saat melakukan prediksi: {e}")
        return {
            'error': str(e),
            'user_id': user_id,
            'prediction': 0,
            'min_prediction': 0,
            'max_prediction': 0,
            'confidence_level': "error"
        }

# Contoh penggunaan (opsional, untuk testing)
if __name__ == "__main__":
    result = predict_spending(
        user_id="sample_user",
        latest_spending=2000000,
        rolling3_spending=1850000,
        target_month=7
    )
    print("Hasil Prediksi:", result)

Error saat memuat model: name '__file__' is not defined
Hasil Prediksi: {'error': 'Model belum dimuat dengan benar', 'user_id': 'sample_user', 'prediction': 0, 'min_prediction': 0, 'max_prediction': 0, 'confidence_level': 'none'}
